### 5. Quick Sort

In this notebook we will be looking at quick sort in details and will be doing the worst and average running time analysis.

In this notebook I will not explaining how quicksort works with examples rather go ahead and implement the bits of quick sort which by itself is self explanatory. The interesting part of the notebook is the proof for correctness and the running time analysis ofthe quicksort

---

Following code gives us the an implementation of in place partitioning of input array. The assumption is that the first element of the list will act as the pivot, before we implement the partition, let us implement a swap function that will swap values of two given index of an array. The partition sub routine apart from partitioning the 



In [147]:
def swap(in_array, ix1, ix2):
    t = in_array[ix1]
    in_array[ix1] = in_array[ix2]
    in_array[ix2] = t

In [148]:
def partition(in_array, start_ix, end_ix):
    #Partitions the input array in place and returns the index of the pivot element
    #in the partitioned array and the number of comparisons performed which is required for the analysis
    #of the number of comparisons performed.
    partition_idx = start_ix
    i = start_ix
    pivot = in_array[partition_idx]
    ncomparisons = 0
    for idx in range(start_ix + 1, end_ix):
        ncomparisons += 1
        if in_array[idx] <=  pivot:
            i += 1
            if idx != i: #Prevent unnecessary swaps                
                t = in_array[i]
                in_array[i] = in_array[idx]
                in_array[idx] = t
    
    swap(in_array, start_ix, i)
    partition_idx = i
    
    return partition_idx , ncomparisons

In [149]:
print('When Pivot is an Intermediate number', partition([3, 2, 1, 5, 0, 7, 8], 0, 7))
print('When Pivot is the Maximum number', partition([8, 2, 1, 5, 0, 7, 3], 0, 7))
print('When Pivot is the Minimum number', partition([0, 2, 1, 5, 8, 7, 3], 0, 7))
print('Input array is same as the example in book', partition([3, 8, 2, 5, 1, 4, 7, 6], 0, 8))

When Pivot is an Intermediate number (3, 6)
When Pivot is the Maximum number (6, 6)
When Pivot is the Minimum number (0, 6)
Input array is same as the example in book (2, 7)



With a working partition function, let us put in place a quick sort function. Ihe impementation is pretty simple and as against merge sort, where we have the merge function after the recursive calls, in quicksort, we perform a linear scan for partition first, followed by the recursive calls on smaller arrays.

In [150]:
def quicksort(in_array, start_ix = 0, end_ix = -1, choose_pivot = lambda x, y, z: y):
    #Quick sort on the input array, the choose_pivot is a function that returns the index
    #of the element in the in_array which will be a pivot. Default strategy is to use the first element as pivot
    end_ix = len(in_array) if end_ix == -1 else end_ix
    if start_ix >= end_ix: #Base case
        return 0

    p_ix = choose_pivot(in_array, start_ix, end_ix)
    #print('Before chosing Pivot', in_array)
    #Swap to set to pivot
    swap(in_array, p_ix, start_ix)
    #print('Before partition', in_array)
    pivot_ix, comparisons = partition(in_array, start_ix, end_ix)
    #print(start_ix, end_ix, pivot_ix, comparisons, in_array)
    left = quicksort(in_array, start_ix, pivot_ix, choose_pivot)
    right = quicksort(in_array, pivot_ix + 1, end_ix, choose_pivot)
    return left + right + comparisons


In [151]:
x = [3, 2, 1, 5, 0, 7, 8]
quicksort(x)
print('Sorting array [3, 2, 1, 5, 0, 7, 8] gives', x)
x = [3, 8, 2, 5, 1, 4, 7, 6]
quicksort(x)
print('Sorting array [3, 8, 2, 5, 1, 4, 7, 6]  gives', x)

Sorting array [3, 2, 1, 5, 0, 7, 8] gives [0, 1, 2, 3, 5, 7, 8]
Sorting array [3, 8, 2, 5, 1, 4, 7, 6]  gives [1, 2, 3, 4, 5, 6, 7, 8]


---

We see above that the sorting function works well. However the efficiency of Quick sort vastly depends on how even the chosen pivot splits in input array into. Thus the choice of Pivot function is extremely crucial for the running time of the Quick sort algorithm. We know Merge sort runs in $O(nlog n)$ time and thats what we aim for. The benefit of quick sort over Merge sort is that it requires no extra memory apart from the constant memory required to store the variables. 
The simplest way to choose a pivot, is to always choose the first element of the input array.

Suppose with this approach, we end up attempting to sort an already sorted array, then by picking up the first element in the list, we do not get a left and a right split but all elements in the right (or left, depending on sorted and sorting order) split except the pivot. This will continue to happen till the entire array is sorted. This is most possible uneven split we can get in Quick sort and we will end up comparing the pivot with all remaining numbers to be sorted, this giving us a worst case time complexity of $O(n^2)$

Another extreme scenario is to implement a choose pivot function that select the median of available values in the left or right side of the array. Assuming that the operation to choose the pivot is a linear time operation (which we will see in next chapter) we can then perform the operation to find the pivot (which is a perfect median of the values we are partioning) and the partition collectively is done in linear time(with high constants, but these are suppressed in Big Oh notations). Thus 

$T(n) \leq 2\cdot T(\frac{n}{2}) + O(n)$

By master method this a divide and conquer algorithm that runs in $O(n log n)$ time. Note that for any other implementation of ``choose pivot`` function, we cannot use master method as the input is not necessarily evenly split. However, when the pivot chosen is guaranteed to be a median of the array being partioned, we are guaranteed an even split.

Thus we have seen how a good ``choose pivot`` functionality decides the runing time of the quick sort algorithm

---

Before we continue with the the analysis of the running time of Quick Sort algorithm, let us, prove the correctness of the algorithm

The correctness is proven by proving the following 2

- Prove the correctness of the partioning routine
- Given the previous, use induction to prove the correctness of the quick sort algorithm

First for the correctness of Quick Sort, the partition subroutine partitions the input array in a way where the pivot ends up in right place for the entire array where all elements on the left of the pivot are less than or equal to the pivot element and the elements on the right are greater than the pivot. With this in mind lets prove the correctness of Quicksort by induction.

For base case, n = 1, Quicksort correctly sorts the input array

The inductive hypothesis is that for k < n, quicksort function p(k) correctly sorts the input. Consider that an input array of size n is given to the quicksort function. The partition function correctly places the pivot in the rightful place and strictly gives us two smaller array of length < n with elements of the left smaller or equal to pivot and on the right, greater than pivot. Since we know by inductive hypothesis, p(k) holds with k < n, quick sort will correctly sort the left and right sub arrays correctly. Since the pivot is already in righ position, the entire array will be sorted.


---

TODO: Give proof for correctness of partition

---

Coming back to the running time of the Quicksort algorithm, we saw that always chosing the first element of the array as pivot will give bad running time of $O(n^2)$ and chosing the pivot by finding the perfect median element in $O(n)$ time will give us a guaranteed $O(nlog n)$ time. However this approach is an overkill and will see if there is an alternate approach which will be fast pivot selection technique (near $O(1)$ time) but gives better running time guarantees than always chosing the first element as pivot.


---

#### Performance of Randomized pivot selection

##### Intuition
Let us assume that the size of the input becomes $\alpha\cdot n$ on each iteration where $\alpha < 0$. To lets assume we hot the base case in k iterations. Thus $\alpha ^ k n = 1$. Taking log on both sizes gives us

$log\alpha^k + logn = 0$ 

$k\cdot log \alpha = -logn$

Let $\beta = \frac{1}{\alpha}$ this $log\alpha = -log\beta$

From above we can say that $k = \frac{logn}{log\beta} = log_{\beta}\alpha$

This makes sense as in case of merge sort where $\alpha = 0.5$, that is when the input array is halved on each recursive call we hit the base case in $log_2n$ iterations. With $\alpha = 0.5$ the value $\beta = 2$

With the above in mind, we know that in case of quick sort taking an exact median is guaranteed to split the array in two exact parts and the running time can be derived using Master method. However, if we are ok with an approximate median, meaning, the split after partition gives us a sub array of size maximum 75% of input array (or a minimum 25% of the input array), then we can say that we would need a maximum of $log_{4/3}n$ levels till we reach the base case. In case of quick sort, the arrays are not overlapping at each level and thus the work done at each level is no more than n. Thus intuitively we have $n\cdot log_{4/3}n$ or $O(nlogn)$ running time

##### Mathematical proof

Let $\Omega$ be a sample space of all possible outcomes of random choices of pivot in Quicksort. This is a well defined random variable because whenever all the pivot element choices are predetermined ($\omega \in \Omega$), QuickSort has a fixed running time RT($\omega$) ranging between $O(nlogn)$ to $O(n^2)$

We then have another random variavble C which is for counting the number of comparisons between two different elements of the array.

One Lemma we we will use is, for a give array A of length $\geq$ 2 and every pivot sequence $\omega$, there is a constant a > 0 such that  $RT(\omega)\: \leq a\cdot C(\omega)$

---

Before we proceed, let us see how to decompose a bigger problem and use linearity of expectation to come up with an expected value for a complex variable

Let Y be the complex random variable expressed a sum of simple random variable X as $Y = \sum_{l = 1}^{m}X_l$

Thus $E[Y] = E[\sum_{l = 1}^{m}X_l] = \sum_{l = 1}^{m}E[X_l]$

If X is an indicator random variable with value 1 and 0 with some probability, the expected value of X is
$E[X_l] = 0\cdot P_r[X_l =0] + 1\cdot P_r[X_l = 1] = P_r[X_l = 1]$

Thus $E[Y] = \sum_{l = 1}^{m}P_r[X_l = 1]$

---

We need to decompose this random variable C into a smaller random variable in a similar way. We need to decompose the random variable C to an indicator random variable. Let is assume $z_i$ denotes the $i{th}$ smallest number in the array. Note that this value is the position of $i{th}$ element in the sorted array and not the input array. For example if 6, 8, 9, 2 are the input values, then $z_1, z_2, z_3\: and\: z_4$ are 2, 6, 8, and 9 respectively

for every $i, j \in 1, 2, 3 \dots n$ where i < j, we define an indicator random variable $X_{ij}$ which denotes whether or not the the $z_i$ and $z_j$ variables are compared or not

$C(\omega)\: = \: \sum_{i = 1}^{n - 1} \sum_{j = i + 1}^{n} X_{ij}$
$E[C(\omega)]\: = \: \sum_{i = 1}^{n - 1} \sum_{j = i + 1}^{n} E[X_{ij}] \: = \sum_{i = 1}^{n - 1}\:  \sum_{j = i + 1}^{n} P_r[X_{ij} = 1]$

One fact to be aware of is that two given values in an array are either compared once or never compared. We will see this why using the following four scenarios. 

With two numbers $z_i$ and $z_j$, following things can happen

- If the chosen pivot is less than $z_i$ the number $z_i$ and $z_j$ (and all numbers between them), both will appear together in the next recursive call
- Similarly, If the chosen pivot is greater than $z_j$ the number $z_i$ and $z_j$ (and all numbers between them), both will appear together in the next recursive call.
- If one number between $z_i$ and $z_j$ are chosen, then $z_i$ and $z_j$ both will appear on different sub arrays on which the quicksort routine is called. Not only would these two number wont be compared now, they would never be compared in any future recursive call.
- One of $z_i$ and $z_j$ is chose, as pivot. In this case they would be compared.

Thus we can see that whether the number $z_i$ and $z_j$ are compared with each other or not is dependent on whether these two numbers and any of the numbers between them is chosen as pivot. From $z_i$ to $z_j$, we have j - i + 1 possible numbers. The probability of $z_i$ or $z_j$ being picked as a pivot between these numbers is therefore $\frac{2}{j - i + 1}$

Coming back to expected value of C we have

$E[C] = \sum_{i = 1}^{n - 1}\sum_{j = i + 1}^{n}\frac{2}{j - i + 1}$

$\sum_{j = i + 1}^{n}\frac{2}{j - i + 1} = 2\cdot (\frac{1}{2} + \frac{1}{3} \dots \frac{1}{n - i + 1})$

Therefore, $E[C] = \sum_{i = 1}^{n - 1}\sum_{j = i + 1}^{n}\frac{2}{j - i + 1} \leq \sum_{i = 1}^{n - 1}\sum_{j = 2}^{n}\frac{2}{j}\: =\: 2(n - 1)\sum_{j = 2}^{n}\frac{1}{j}$

What is the value of $\sum_{j = 2}^{n}\frac{1}{j}$

The value $\sum_{j = 2}^{n}\frac{1}{j}\: \leq \:\int_1^n \frac{1}{x}\cdot dx \: = \: ln\:\vert_1^n  = ln(n) - ln0 = ln(n)$

Therefore

$E[C] = 2(n - 1)\sum_{j = 2}^{n}\frac{1}{j} \: \leq\: 2(n - 1)\cdot ln(n)$. Therefore running time of Quicksort is $O(nlogn)$

---

TODO: Work on the optional sectctional to prove that no comparison based sorting algorithm do better than $O(nlogn)$. I am yet to convince myself with the proof and not everything is intuitive as of yet to me.

---




### Test Your Understanding

---

#### Problem 5.1

Suppose are are told that the array 3, 1, 2, 4, 5, 8, 7, 6, 9 is partioned, then the value 4 and 5 are possible pivot elements as they are in the rightful position of the array and additionally all elements to the left are smaller and on the right are greater than them.

---

#### Problem 5.2

Suppose $\alpha$ is a number between 0 and 0.5, then what is the probability that for a randomly chosen pivot, the size of each sub problem is  at least $\alpha$ times n.

To get an two arrays of size minimum $\alpha$ times n, the number chosen as pivot has to be greater than the $\alpha^{th}$ order statistic, but no greater than $(1 - \alpha)n^{th}$ order statistic.  We have total of $2\alpha n$ numbers ($\alpha n + (n - n(1 - \alpha)) = 2\alpha n$) whose value is less than $\alpha^{th}$ and greater than $n(1 - \alpha)^{th}$ statistic. Thus the probability that the array size is less than $\alpha n$ is $2\alpha$ and thus the probability that the size of atleast $\alpha n$ is $1 - 2\alpha$

 
---

#### Problem 5.3

Let $\alpha$ be some constant independent of n and between 0 and 0.5. Whenever we make a recursive call on an array of length k, we get sub array with length between $\alpha k$ and $(1 - \alpha)k$.

How many recursive calls are needed before we hit the base case. 

For an array of size k that is shrinking at a rate of $\alpha$, we will need $-log_{\alpha}k$ iteration. This is something we have seen previously. The sign negative is needed as log to the base of a number less than 1 is negative and the number of iterations have to be positive. Now $-log_{\alpha}k = -\frac{ln\:k}{\ln \alpha}$, thus we will require between $-\frac{ln\:k}{\ln \alpha}$ and $-\frac{ln\:k}{\ln (1 - \alpha)}$ iterations

---

#### Problem 5.4

What is the maximum and minimum possible tree depth of recursive calls to an array of size n.

The best case is when each pivot is the true median of the input array, in which case on each iteration we split the array in exact halves and thus the best possible tree depth we can expect is $O(\log n)$, on other hand the pivot chosen splits the array in the work possible way with all elements ending up on one side, in which case the maximum possible length of the tree is $O(n)$

---

#### Problem 5.5

I currently am clueless on this derivation.

---

#### Problem 5.6

To facilitate reading numbers from a file, we will implement a function which given the file name returns a list of integers

In [152]:
def to_int_array(fileName):
    with open(fileName, 'r') as f:
        lines = f.readlines()
        return [int(line.strip()) for line in lines]

In [153]:
def median_of_3_index(array, start_idx, end_idx):
    #for an between start_idx and end_idx, find the element sitting in between these and then return the 
    #index of the number whose value is a median of the thee value
    s_val = array[start_idx]
    e_val = array[end_idx]
    m_idx = start_idx + (end_idx - start_idx) // 2
    m_val = array[m_idx]
    max_v = max([s_val, e_val, m_val])
    min_v = min([s_val, e_val, m_val])

    return start_idx if s_val != max_v and  s_val != min_v else end_idx if e_val != max_v and e_val != min_v else m_idx
    
    
prob56t1 = to_int_array('problem5.6test1.txt')
#Quick sort with first number as pivot
numcomp_p1 = quicksort(prob56t1)
prob56t1 = to_int_array('problem5.6test1.txt')
numcomp_p2 = quicksort(prob56t1, choose_pivot = lambda x, y, z: z - 1)
prob56t1 = to_int_array('problem5.6test1.txt')
numcomp_p3 = quicksort(prob56t1, choose_pivot = lambda x, y, z: median_of_3_index(x, y, z - 1))
print('Test 1: Num comparisons with First, Last and Median of 3 are',numcomp_p1, numcomp_p2, numcomp_p3, 'respectively')

prob56t2 = to_int_array('problem5.6test2.txt')
#Quick sort with first number as pivot
numcomp_p1 = quicksort(prob56t2)
prob56t2 = to_int_array('problem5.6test2.txt')
numcomp_p2 = quicksort(prob56t2, choose_pivot = lambda x, y, z: z - 1)
prob56t2 = to_int_array('problem5.6test2.txt')
numcomp_p3 = quicksort(prob56t2, choose_pivot = lambda x, y, z: median_of_3_index(x, y, z - 1))
print('Test 2: Num comparisons with First, Last and Median of 3 are',numcomp_p1, numcomp_p2, numcomp_p3, 'respectively')

prob56 = to_int_array('problem5.6.txt')
#Quick sort with first number as pivot
numcomp_p1 = quicksort(prob56)
prob56 = to_int_array('problem5.6.txt')
numcomp_p2 = quicksort(prob56, choose_pivot = lambda x, y, z: z - 1)
prob56 = to_int_array('problem5.6.txt')
numcomp_p3 = quicksort(prob56, choose_pivot = lambda x, y, z: median_of_3_index(x, y, z - 1))
print('Problem: Num comparisons with First, Last and Median of 3 are',numcomp_p1, numcomp_p2, numcomp_p3, 'respectively')

Test 1: Num comparisons with First, Last and Median of 3 are 25 31 21 respectively
Test 2: Num comparisons with First, Last and Median of 3 are 620 573 502 respectively
Problem: Num comparisons with First, Last and Median of 3 are 162085 164123 138382 respectively
